## Mathematica code to calculate Non-normalized Bakry-Emery-Ricci Curvature

In [1]:
(*------------------------------------------------------------------------------------------------------------------------*)
(* Function "BERCurvature" calculates Bakry-Emery curvature for Undirected and Unweighted networks from a given edgelist
Modified version of the code provided in the following url: https://sites.google.com/view/florentin-muench/graph-curvature?authuser=0 *)
(*------------------------------------------------------------------------------------------------------------------------*)

(* Creates Laplacian of the network from a given adjacency matrix*)
AdjToLaplace[A_] := A - DiagonalMatrix[A.SparseArray[Table[1,Length[A]]]];

BERCurvature[EdgeList_] := Module[{BERCurv},
    (* Creates graph from the edgelist*)
    graph = Graph[UndirectedEdge @@@ EdgeList];
    Print[graph];
    (* Creates adjacency matrix from the graph*)
    adjacencyMatrix = AdjacencyMatrix[graph];
    
    (* Creates nodelist from the edgelist*)
    nodeList = VertexList[graph];
    
    (* Call the function "AdjToLaplace" *)
    L = AdjToLaplace[SparseArray[Normal[adjacencyMatrix]]];
    (* Non-zero positions for pairs with shortest path length = 2 *)
    L2=L.L;

    (* Removes all the selfloops*)        
    A=SparseArray[L-DiagonalMatrix[Diagonal[L]]]; 
    VV=Length[L];

    (* Store the nodes with shortest path length <= 2 *)
    b2all=Table[Flatten[L2[[k]]["NonzeroPositions"]],{k,VV}];

    (* Store the neighbouring nodes *)
    b1=Table[Flatten[A[[k]]["NonzeroPositions"]],{k,VV}];
        
    (* Consider the nodes with shortest path length = 2 *)
    b2=Table[Complement[b2all[[k]],b1[[k]],{k}],{k,VV}];

    (* 3-D sparse array with diagonal = 1 *)
    fg=SparseArray[Table[{i,i,i}->1,{i,VV}]];

    (* transposes a 3-D matrix 'ten' by permuting its dimensions 2 and 3 *)
    T[ten_]:=TensorTranspose[ten,{1,3,2}];

    (* Construction of the Gamma operator *)
    Gam=(L.fg - fg.L - T[fg.L]); 

    (* Construction of the Gamma 2 operator *)
    Gam2=(L.Gam - Gam.L  - T[Gam.L])/2; 

    inv := Compile[{{x, _Real}},If[x!=0,1/x,0]]; 
    (* Gives Reciprocal of x *)
    Inv[x_]:=Map[inv,x,{2}] ;   

    (* Construct a matrix with neighbours of the nodes *)
    Q[k_,b1_,b2_,G2_]:=If[b2=={},G2[[b1,b1]],G2[[b1,b1]]-G2[[b1,b2]].Inv[G2[[b2,b2]]] . G2[[b2,b1]]];
    (* Returns the curvature (minimum eigenvalue of ) of the node k *)
    Ricc[k_]:=If[b1[[k]]=={},0,Min[Eigenvalues[{Q[k,b1[[k]],b2[[k]],Gam2[[k]]],Gam[[k]][[b1[[k]],b1[[k]]]]}]]];

    (*Ric gives the list of the Bakry Emery curvature for every node*)
    Ric=Table[Ricc[k],{k,VV}];
    BERCurv=Transpose[{nodeList, Ric}];
    BERCurv
    ]

### Calculates Bakry-Emery curvature for model networks

In [159]:
model = "BA";
parameter = "1000_3";
i = 1;

filepath = "../Data/Model_Networks/"<> ToString[model] <>"/"<> ToString[parameter] <>"/"<> ToString[model] <>"_edge_seed_"<> ToString[i] <>".txt"; 
edgeList = Import[filepath, "Table"];

(* Call the function "BERCurvature" *)
BERC = BERCurvature[edgeList];

path = "../Data/Model_Networks/"<> ToString[model] <>"/"<> ToString[parameter] <>"/BakryEmery";
If[! DirectoryQ[path],
  CreateDirectory[path]];
  
(* Save the BERC values in a .txt file *)
Export["../Data/Model_Networks/"<> ToString[model] <>"/"<> ToString[parameter] <>"/BakryEmery/"<> ToString[model] <>"_node_seed_"<> ToString[i] <>".txt", BERC,"Table"]; 

Graph[<1000>, <3000>]


### Calculates Bakry-Emery curvature for real-world networks

In [ ]:
filepath = "../Data/Real_Networks/ArenasEmail/ArenasEmail_Edge.txt";
edgeList = Import[filepath, "Table"];
BERC = BERCurvature[edgeList];

(* Save the BERC values in a .txt file *)
Export["../Data/Real_Networks/ArenasEmail2/ArenasEmail_BakryEmery_node.txt",  BERC,"Table"]; 